# Massive Setup Guide (formerly Polygon.io)

**Provider:** Massive (Real-time & historical market data)

**Best for:** Professional-grade data, real-time quotes, options data

**Time:** 5 minutes

**Cost:** FREE tier available for different asset classes


## Step 1: Create Massive Account

1. Go to [massive.com](https://massive.com/)
2. Sign up

## Step 2: Get Your API Key

1. Go to your Massive dashboard
2. Your API key is shown on the dashboard
3. **Copy the key**




## Step 3: Set Your API Key

Run this cell and paste your API key when prompted:

In [1]:
import os
from getpass import getpass

# Prompt for API key (hidden input)
api_key = getpass("Enter your Massive API key: ")
os.environ["MASSIVE_API_KEY"] = api_key

print("✅ API key set!")
print(f"   Key format: {api_key[:8]}...{api_key[-4:]}")

Enter your Massive API key: ··········
✅ API key set!
   Key format: 2ELdmHzC...nppr


## Step 4: Install Massive Package

In [3]:
!pip install -q massive pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 6.2 MB/s eta 0:00:00


## Step 5: Test Your Connection - Previous Close

In [4]:
from massive import RESTClient

client = RESTClient(os.environ["MASSIVE_API_KEY"])

# Get previous day's close
aggs = client.get_previous_close_agg("AAPL")

print("✅ Connection successful!")
for agg in aggs:
    print(f"   Ticker: {agg.ticker}")
    print(f"   Close: ${agg.close}")
    print(f"   Volume: {agg.volume:,}")
    print(f"   VWAP: ${agg.vwap:.2f}")

✅ Connection successful!
   Ticker: AAPL
   Close: $247.65
   Volume: 54,933,217.0
   VWAP: $247.71


## Step 6: Get Historical Aggregates

In [5]:
import pandas as pd
from datetime import datetime, timedelta

# Get daily bars for last 30 days
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

aggs = client.get_aggs(
    ticker="AAPL",
    multiplier=1,
    timespan="day",
    from_=start_date.strftime("%Y-%m-%d"),
    to=end_date.strftime("%Y-%m-%d")
)

# Convert to DataFrame
data = [{
    "date": datetime.fromtimestamp(agg.timestamp / 1000),
    "open": agg.open,
    "high": agg.high,
    "low": agg.low,
    "close": agg.close,
    "volume": agg.volume,
    "vwap": agg.vwap
} for agg in aggs]

df = pd.DataFrame(data)
df.set_index("date", inplace=True)

print("✅ Historical data retrieved!")
print(f"   Period: {df.index[0].date()} to {df.index[-1].date()}")
print(f"   Records: {len(df)}")
print(f"\nLast 5 days:")
print(df.tail())

✅ Historical data retrieved!
   Period: 2025-12-23 to 2026-01-21
   Records: 19

Last 5 days:
                       open    high     low   close      volume      vwap
date                                                                     
2026-01-14 05:00:00  259.49  261.82  256.71  259.96  40019421.0  259.2335
2026-01-15 05:00:00  260.65  261.04  257.05  258.21  39388564.0  258.9417
2026-01-16 05:00:00  257.90  258.90  254.93  255.53  72142773.0  256.2442
2026-01-20 05:00:00  252.73  254.79  243.42  246.70  80110363.0  248.1480
2026-01-21 05:00:00  248.70  251.56  245.18  247.65  54933217.0  247.7120


## Step 7: Get Ticker Details

In [6]:
# Get company details
details = client.get_ticker_details("AAPL")

print("✅ Ticker details retrieved!")
print(f"   Name: {details.name}")
print(f"   Market: {details.market}")
print(f"   Locale: {details.locale}")
print(f"   Primary Exchange: {details.primary_exchange}")
print(f"   Type: {details.type}")
print(f"   Currency: {details.currency_name}")
print(f"   Market Cap: ${details.market_cap/1e9:.1f}B" if details.market_cap else "   Market Cap: N/A")
print(f"   Employees: {details.total_employees:,}" if details.total_employees else "   Employees: N/A")

✅ Ticker details retrieved!
   Name: Apple Inc.
   Market: stocks
   Locale: us
   Primary Exchange: XNAS
   Type: CS
   Currency: usd
   Market Cap: $3639.9B
   Employees: 166,000


## Step 8: Get Latest Quote (Real-time)

In [7]:
# Get latest trade (requires paid tier for real-time)
# Free tier gets 15-min delayed data
try:
    quote = client.get_last_trade("AAPL")
    print("✅ Latest trade retrieved!")
    print(f"   Price: ${quote.price}")
    print(f"   Size: {quote.size}")
    print(f"   Exchange: {quote.exchange}")
except Exception as e:
    print(f"⚠️ Real-time data requires paid tier")
    print(f"   Free tier provides previous close data")

⚠️ Real-time data requires paid tier
   Free tier provides previous close data


## Step 9: Create a Tool for smolagents

In [8]:
!pip install -q smolagents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 3.8 MB/s eta 0:00:00


In [9]:
from smolagents import tool

@tool
def get_stock_price_massive(ticker: str) -> dict:
    """Get previous day's stock data from Massive.

    Args:
        ticker: Stock symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')

    Returns:
        Dictionary with open, high, low, close, volume, and VWAP
    """
    from massive import RESTClient
    import os

    client = RESTClient(os.environ["MASSIVE_API_KEY"])
    aggs = client.get_previous_close_agg(ticker.upper())

    for agg in aggs:
        return {
            "ticker": agg.ticker,
            "open": agg.open,
            "high": agg.high,
            "low": agg.low,
            "close": agg.close,
            "volume": agg.volume,
            "vwap": round(agg.vwap, 2),
            "source": "Massive"
        }

    return {"error": "No data found", "ticker": ticker}

@tool
def get_company_info_massive(ticker: str) -> dict:
    """Get company information from Massive.

    Args:
        ticker: Stock symbol (e.g., 'AAPL', 'MSFT')

    Returns:
        Dictionary with company name, market cap, employees, etc.
    """
    from massive import RESTClient
    import os

    client = RESTClient(os.environ["MASSIVE_API_KEY"])
    details = client.get_ticker_details(ticker.upper())

    return {
        "ticker": ticker.upper(),
        "name": details.name,
        "market": details.market,
        "exchange": details.primary_exchange,
        "market_cap": details.market_cap,
        "employees": details.total_employees,
        "description": details.description[:200] + "..." if details.description else None,
        "source": "Massive"
    }

# Test the tools
print("Tool 1 - Stock Price:")
print(get_stock_price_massive("MSFT"))

print("\nTool 2 - Company Info:")
print(get_company_info_massive("MSFT"))

Tool 1 - Stock Price:
{'ticker': 'MSFT', 'open': 452.595, 'high': 452.69, 'low': 438.68, 'close': 444.11, 'volume': 37939952.0, 'vwap': 445.2, 'source': 'Massive'}

Tool 2 - Company Info:
{'ticker': 'MSFT', 'name': 'Microsoft Corp', 'market': 'stocks', 'exchange': 'XNAS', 'market_cap': 3300793240362.0503, 'employees': 228000, 'description': 'Microsoft develops and licenses consumer and enterprise software. It is known for its Windows operating systems and Office productivity suite. The company is organized into three equally sized broad s...', 'source': 'Massive'}


## ✅ Setup Complete!

You're ready to use Massive in the course notebooks.
**Next steps:**
- Save your API key securely (password manager)
- Remember: 5 calls/min on free tier
- Explore the [Massive documentation](https://massive.com/docs/stocks/getting-started)
- Continue to the course Module 1 notebook
